In [9]:
import networkx as nx
import json
import pandas as pd

edgefile = "../../data/graph_toronto_knn_20.csv"
restfile = "../../data/yelp_toronto.csv"
communityfile = "../../data/louvain_dict_knn_20.json"

In [7]:
# read in files, construct graph
edges = pd.read_csv(edgefile, ' ', header = 0)    
df = pd.read_csv(restfile, sep=' ')
G = nx.from_pandas_edgelist(edges, source = 'r1', target='r2')


In [51]:
from collections import Counter
cats = []
for row in df['categories']:
    row = row.replace(" ", "")
    cats += row.split(',')

ctr = dict(Counter(cats))
print sorted(ctr.keys(), key=lambda x: ctr[x], reverse=True)

# "Coffee & Tea", "Bars", "Sandwiches", "Breakfast & Brunch", "Chinese", "Middle Eastern", "Japanese", "Pizza", "Mexican", "Mediterranean", "Korean", "Thai"

['Restaurants', 'Food', 'Coffee&Tea', 'Nightlife', 'Bars', 'SpecialtyFood', 'Sandwiches', 'Canadian(New)', 'Breakfast&Brunch', 'Cafes', 'Chinese', 'Italian', 'Bakeries', 'Pizza', 'Japanese', 'Desserts', 'FastFood', 'Burgers', 'SushiBars', 'American(Traditional)', 'Grocery', 'Indian', 'Mediterranean', 'MiddleEastern', 'AsianFusion', 'Thai', 'EventPlanning&Services', 'IceCream&FrozenYogurt', 'Mexican', 'Pubs', 'Korean', 'Seafood', 'JuiceBars&Smoothies', 'ChickenWings', 'Salad', 'Caterers', 'Vietnamese', 'Barbeque', 'Wine&Spirits', 'Beer', 'Vegetarian', 'French', 'Diners', 'ComfortFood', 'Caribbean', 'Shopping', 'Greek', 'American(New)', 'Vegan', 'Lounges', 'CocktailBars', 'Halal', 'EthnicFood', 'WineBars', 'Gluten-Free', 'Arts&Entertainment', 'FoodDeliveryServices', 'Delis', 'HealthMarkets', 'SportsBars', 'TeaRooms', 'Gastropubs', 'Steakhouses', 'Tapas/SmallPlates', 'TapasBars', 'Soup', 'ConvenienceStores', 'LatinAmerican', 'MeatShops', 'Pakistani', 'Chocolatiers&Shops', 'Portuguese', 'I

In [10]:
#community dicts
with open(communityfile, "r") as f:
    assignments = json.loads(f.read())
partition = {}
#dict from community # --> list of nodes in community
for part, idx in assignments.iteritems():
    if idx not in partition:
        partition[idx] = [part]
    else:
        partition[idx].append(part)

In [20]:
df['review_count_weighted'] = df['review_count'] * df['stars'] / 5.0

In [21]:
print df['review_count_weighted']

0         5.6
1       116.8
2        18.4
3        17.5
4        84.0
5         2.1
6        24.8
7         7.2
8        35.7
9         2.0
10        2.4
11       22.8
12        2.8
13       96.8
14        2.1
15        3.2
16        4.0
17       17.5
18       24.3
19        8.0
20        3.0
21       10.8
22       19.2
23        5.6
24       20.0
25        5.0
26       11.4
27       20.4
28        7.0
29      119.7
        ...  
9647     33.6
9648     15.0
9649      3.0
9650      0.9
9651      6.6
9652      7.0
9653     35.2
9654      8.4
9655      4.8
9656      3.0
9657     10.2
9658      1.2
9659     12.0
9660      8.5
9661      8.1
9662     22.8
9663      4.9
9664      5.4
9665      3.0
9666      9.8
9667     88.9
9668     56.8
9669      1.5
9670      8.0
9671      1.2
9672      6.5
9673      8.1
9674      2.4
9675    189.9
9676      1.8
Name: review_count_weighted, Length: 9677, dtype: float64


In [58]:
#add features to df!
##### degree #####
df['degree'] = df['id'].map(lambda x: 0 if x not in assignments else float(G.degree(x)))

##### clustering coefficient #####
df['clustering'] = df['id'].map(lambda x: 0 if x not in assignments else nx.clustering(G, x))

##### edge density of community #####
comm_edge_densities = {}
for idx, comm in partition.iteritems():
    H = G.subgraph(comm)
    comm_edge_densities[idx] = nx.density(H)

df['comm_edge_density'] = df['id'].map(lambda x: 0 if x not in assignments else comm_edge_densities[assignments[x]])
    
##### community size #####
df['comm_sz'] = df['id'].map(lambda x: 0 if x not in assignments else len(partition[assignments[x]]))

##### avg community rating #####
#precalculate avg community ratings
comm_review_counts = {}
for idx, comm in partition.iteritems():
    metric_sum = 0.0
    for node in comm:
        node_row = df.loc[df['id'] == node]
        metric_sum += float(node_row['review_count'])
    metric_sum /= len(comm)
    comm_review_counts[idx] = metric_sum
        
df['comm_review_count'] = df['id'].map(lambda x: 0 if x not in assignments else comm_review_counts[assignments[x]])
#predicting: review count, rating separately, review count * normalized rating? 

In [59]:
print df

                          id   latitude  longitude  \
0     9A2quhZLyWk0akUetBd8hQ  43.664378 -79.414424   
1     tZnSodhPwNr4bzrwJ1CSbw  43.664125 -79.411886   
2     5J3b7j3Fzo9ISjChmoUoUA  43.681328 -79.427884   
3     PMDlKLd0Mxj0ngCpuUmE5Q  43.670885 -79.392379   
4     YIez_A3WOt9J2SXN7OMa2Q  43.745928 -79.324623   
5     xuUzASHWjRJRFv6Ck5pO7g  43.649972 -79.383223   
6     mr3rQcYBKWu2L6o7qtQ9Wg  43.669116 -79.426021   
7     Q_cfbLdAxkLiEZW5TO5T6A  43.595149 -79.529977   
8     cuXCQM-9VwpZlSneEY1b3w  43.708002 -79.375814   
9     hsWx7ya8jLMhi8ZWX23Thg  43.706983 -79.396499   
10    DcyeRzICLrMkrPpJDzjQ6Q  43.674164 -79.287392   
11    i2Fd0dl39BZ8nVxBnSPsKg  43.691085 -79.575587   
12    2j5hnwYzS-YLJn6IFqloCw  43.648678 -79.397234   
13    _T8qy9XAKAFLJdmoLg1Q-g  43.772992 -79.414052   
14    FI6XxleZ6Z1xJ9BS4zLKgQ  43.774374 -79.414043   
15    Bw_rDtU2REt7ZlkPWT21kg  43.742019 -79.591076   
16    2tg7rpwAbrex81bMKOnOtQ  43.699097 -79.359723   
17    vwVoksbTKehDP8ZGUQaVfQ

In [60]:
for col in df.columns:
    print len(df[col])

9677
9677
9677
9677
9677
9677
9677
9677
9677
9677
9677


In [76]:
from sklearn.lineaar_model import LogisticRegression
import numpy as np


In [96]:
##### degree #####
df['degree'] = df['id'].map(lambda x: 0 if x not in assignments else float(G.degree(x)))

# for node in df['id']:
#     print node
#     int(G.degree(node))
# # print df.degree.unique()

In [56]:
import networkx as nx
import json
import os, sys
sys.path.append('../clustering')
import knn
import evaluate
import cutoff

# sys.path.append('src/')
# from knn import knn, split
# from evaluate import get_point_info
# from cutoff import filter_connected_components

k = 8
city = "toronto"
categories = ["Coffee & Tea", "Bars", "Sandwiches", "Breakfast & Brunch", "Chinese", "Middle Eastern", "Japanese", "Pizza", "Mexican", "Mediterranean", "Korean", "Thai"]
# categories = ["Coffee & Tea"]
output_file = "../../data/knnsplit/graph_" + city + "_cats_uncut_8.csv"
node_file = "../../data/yelp_" + city + ".csv"

mapping = {}
(coords, nids, category_map, nodes) = evaluate.get_point_info(node_file)
with open(output_file.format(output_file), 'w+') as fout:
    fout.write('r1 r2 weight\n')
    end_idx = 0
    for category in categories:
        graph = knn.knn(nodes, k)
        graph = knn.split(graph, category, category_map)
        # graph = cutoff.filter_connected_components(graph)
        nx.write_edgelist(graph, fout, data="weight")
        
       
        for i, component in enumerate(nx.connected_components(graph)):
            for node in component:
                mapping[node] = i + end_idx
        end_idx += nx.number_connected_components(graph)

    dict_file = "../../data/knnsplit/community_" + city + "_cats_uncut_8.json"
    with open(dict_file, "w+") as f:
        json.dump(mapping, f)

In [55]:
print mapping

{'KN7gZcMIfFUhBKcIGt2XkQ': 317, 'g5agBkB1d4YAK3rotmz2pQ': 286, '_oL8XX8dusHBNYQyDhF4fA': 354, 'yuwi6SLaY-5OmcW2WsUZjA': 318, 'P3ndcRpXSpDNucOqFwcx2g': 1, 'DELT8o16Fa3Ss0ulmOvVdw': 229, 'r92duI6MtSi1-seaIeQUWA': 399, 'V84XKSe9gMmky0sh1If_dA': 372, 'TJbGgH70MDdDn0LZnlwWIg': 367, 'TWa07Bllw_jfI2Y2Lry9DQ': 188, 'l0Vk2cI6TQ1RBm59rUaWWA': 27, 'OlfcPlE-g4qGm_4V0RlpGw': 383, 'mTPx7c-3F-LkqJyuF6Eylg': 307, 'j8gdbeZoPwjeh11BKzvrNg': 217, 'ulOA7pJjIgrtp3ma3iPesA': 230, 'vl5P6HeohnND5VmXc666Tw': 399, 'VqLbHAeddYcnj99OD_z2zA': 232, 'U7RnIJ6HGjixl1VwRTH2RQ': 408, 'Tt7syUJHEuONVjczXnkEuQ': 5, 'Kjg1v-9P_32QuttjRR5z1w': 11, 'c4qE3ygdEQUD9dZjganavA': 89, 'kZYWDd86nDoOqXPKlYn5sw': 137, 'KtgDHvAGwZckZgctRcwjNQ': 122, 'HfQb9_TMWptbvnb550Kgmw': 130, '9MfAMuc_f_dxqRgOxmF7ow': 191, 'wDC6JwUz-3yO_SbqH0Z6Hw': 310, 'A7xpCA_PF7oY5W60NaqvHQ': 40, 'uTZlPGcaueT0nAXioAeI-Q': 368, '4Qk041pJRyQb2SkEGQiTxw': 227, 'UAHc6MD-3N4bNk5d-wFtRw': 86, 'lBRtpBISWIJZV1mzvmvVUA': 436, 'BVW93go7fyqTszxx2h8XTg': 329, 'OTE_y7dsOTIw-S7